Dataset and related content taken from Natural Language Processing for Mobile App Privacy Compliance. Peter Story, Sebastian Zimmeck, Abhilasha Ravichander, Daniel Smullen, Ziqi Wang, Joel Reidenberg, N. Cameron Russell, and Norman Sadeh. AAAI Spring Symposium on Privacy Enhancing AI and Language Technologies (PAL 2019), Mar 2019.

Available here – APP350: https://usableprivacy.org/data

EDA should be in the context of my data.  I should state what I expect BEFORE checking for it in the data.

Also at the top I can include links, context, goals

In [1]:
import pandas as pd
from pandas import json_normalize
import yaml
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

import sys
from collections import defaultdict
from collections import Counter

import ds_utils_callum
import priv_policy_manipulation_functions as priv_pol_funcs

Put the first policy into a df

## Populating top-level df

In [2]:
all_policies_df = priv_pol_funcs.load_all_policies()
all_policies_df.head(5)

,policy_id,policy_name,policy_type,contains_synthetic,segments
0,1,6677G,TEST,False,"[{'segment_id': 0, 'segment_text': 'PRIVACY PO..."
1,2,AIFactory,TEST,False,"[{'segment_id': 0, 'segment_text': 'AI Factory..."
2,3,AppliqatoSoftware,TEST,False,"[{'segment_id': 0, 'segment_text': 'Automatic ..."
3,4,BandaiNamco,TEST,False,"[{'segment_id': 0, 'segment_text': 'MOBILE APP..."
4,5,BarcodeScanner,TEST,False,"[{'segment_id': 0, 'segment_text': 'Skip to co..."


In [3]:
all_policies_df = priv_pol_funcs.add_metadata_to_policy_df(all_policies_df)
all_policies_df.head(5)

,policy_id,policy_name,policy_type,contains_synthetic,segments,num_segments,num_annotated_segs,total_characters
0,1,6677G,TEST,False,"[{'segment_id': 0, 'segment_text': 'PRIVACY PO...",36,11,12703
1,2,AIFactory,TEST,False,"[{'segment_id': 0, 'segment_text': 'AI Factory...",14,5,5995
2,3,AppliqatoSoftware,TEST,False,"[{'segment_id': 0, 'segment_text': 'Automatic ...",8,1,2450
3,4,BandaiNamco,TEST,False,"[{'segment_id': 0, 'segment_text': 'MOBILE APP...",57,14,32323
4,5,BarcodeScanner,TEST,False,"[{'segment_id': 0, 'segment_text': 'Skip to co...",32,3,6667


# Making segments

Now to make a new dataframe where each row represents a paragraph (segment).

First I will get this to work for a single policy. Then I will loop through all the policies to apply the same manipulation.

In [4]:
all_segments_df = priv_pol_funcs.generate_segment_df(all_policies_df)
all_segments_df.head()

,source_policy_number,policy_segment_id,segment_text,annotations,sentences
segment_id,,,,,
0,1,0,PRIVACY POLICY This privacy policy (hereafter ...,[],[]
1,1,1,1. ABOUT OUR PRODUCTS 1.1 Our products offer a...,[],[]
2,1,2,2. THE INFORMATION WE COLLECT The information ...,[{'practice': 'Identifier_Cookie_or_similar_Te...,"[{'sentence_text': 'IP ADDRESS, COOKIES, AND W..."
3,1,3,"2.2 In addition, we store certain information ...",[{'practice': 'Identifier_Cookie_or_similar_Te...,"[{'sentence_text': '6677g may use cookies, web..."
4,1,4,(c) to remember your preferences and registrat...,[],[]


In [6]:
all_segments_df.shape

(15543, 5)

In [7]:
all_segments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15543 entries, 0 to 15542
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   source_policy_number  15543 non-null  int64 
 1   policy_segment_id     15543 non-null  int64 
 2   segment_text          15543 non-null  object
 3   annotations           15543 non-null  object
 4   sentences             15543 non-null  object
dtypes: int64(2), object(3)
memory usage: 607.3+ KB


# Next step of extraction

I probably want to separate it out to the sentence level, as that is the max granularity of the annotations and some paragraphs are just one sentence anyway.

In [8]:
all_segments_df.loc[2,:]

source_policy_number                                                    1
policy_segment_id                                                       2
segment_text            2. THE INFORMATION WE COLLECT The information ...
annotations             [{'practice': 'Identifier_Cookie_or_similar_Te...
sentences               [{'sentence_text': 'IP ADDRESS, COOKIES, AND W...
Name: 2, dtype: object

In [9]:
all_segments_df.loc[2, 'segment_text']

"2. THE INFORMATION WE COLLECT The information that our products collect includes (among others) the following: A) IP ADDRESS, COOKIES, AND WEB BEACONS 2.1 When you visit our products, our servers automatically save your computer's IP address. IP addresses will be collected, along with information about the actual web pages that you visit on our products. If you arrive at our products via a link from another product, the URL of the linking product and the URL of any product that you link to next will also be collected."

In [10]:
all_segments_df.loc[2, 'annotations']

[{'practice': 'Identifier_Cookie_or_similar_Tech_1stParty',
  'modality': 'PERFORMED'},
 {'practice': 'Identifier_IP_Address_1stParty', 'modality': 'PERFORMED'}]

In [11]:
all_segments_df.loc[2, 'sentences']

[{'sentence_text': 'IP ADDRESS, COOKIES, AND WEB BEACONS',
  'annotations': [{'practice': 'Identifier_Cookie_or_similar_Tech_1stParty',
    'modality': 'PERFORMED'},
   {'practice': 'Identifier_IP_Address_1stParty', 'modality': 'PERFORMED'}]},
 {'sentence_text': 'IP addresses will be collected, along with information about the actual web pages that you visit on our products.',
  'annotations': [{'practice': 'Identifier_IP_Address_1stParty',
    'modality': 'PERFORMED'}]},
 {'sentence_text': 'The information that our products collect includes (among others) the following:',
  'annotations': [{'practice': 'Identifier_Cookie_or_similar_Tech_1stParty',
    'modality': 'PERFORMED'},
   {'practice': 'Identifier_IP_Address_1stParty', 'modality': 'PERFORMED'}]},
 {'sentence_text': "When you visit our products, our servers automatically save your computer's IP address.",
  'annotations': [{'practice': 'Identifier_IP_Address_1stParty',
    'modality': 'PERFORMED'}]}]

In [12]:
all_segments_df.loc[3, 'segment_text']

'2.2 In addition, we store certain information from your browser, using "cookies." A cookie is a piece of data stored on the user\'s computer and is tied to information about the user. 6677g may use cookies, web beacons (web bugs), or similar technologies to enhance and personalize your experience on our products, including the following: (a) to operate and improve offerings on our products; (b) to help authenticate you when you are on our products;'

In [13]:
all_segments_df.loc[4, 'segment_text']

'(c) to remember your preferences and registration information, as applicable;'

In [14]:
all_segments_df.loc[5, 'segment_text']

'(d) to present and help measure and research the effectiveness of 6677g offerings, advertisements, and email communications; and'

In [15]:
all_segments_df.loc[6, 'segment_text']

'(e) to customize the content and advertisements provided to you through our products.'

In [16]:
all_segments_df.loc[7, 'segment_text']

'2.3 6677g may also use ad network providers to help present advertisements on our products. These ad network providers use cookies, web beacons, or similar technologies to help the presenting, better targeting, and measuring of the effectiveness of their advertisements, using data gathered over time and across their networks of web pages to determine or predict the characteristics and preferences of their audiences. 6677g offers some services in connection with other products. Personal information that you provide to those sites may be sent to 6677g in order to deliver these services. 6677g processes such information in accordance with this Privacy Policy.'

I probably want to separate it out to the sentence level, as that is the max granularity of the annotations and some paragraphs are just one sentence anyway.

## Extracting list of practices

In [17]:
with open("APP_350_v1_1/features.yml", "r") as stream:
    try:
        features_yml = (json_normalize(yaml.safe_load(stream)))
    except yaml.YAMLError as exc:
        print(exc)

In [18]:
features_yml

,data_types,modalities.PERFORMED,modalities.NOT_PERFORMED,parties.FirstParty,parties.ThirdParty
0,"[{'practices': ['Contact_1stParty', 'Contact_3...","[consent, permission, opt, collect, access, g...","[not collect, no longer collect, not access, n...","[ we , you , us , our , the app, the software]","[partner, third part, third-part, service prov..."


In [20]:
data_types = json_normalize(features_yml['data_types'])

In [21]:
data_types

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,"{'practices': ['Contact_1stParty', 'Contact_3r...",{'practices': ['Contact_Address_Book_1stParty'...,"{'practices': ['Contact_City_1stParty', 'Conta...",{'practices': ['Contact_E_Mail_Address_1stPart...,"{'practices': ['Contact_Password_1stParty', 'C...",{'practices': ['Contact_Phone_Number_1stParty'...,{'practices': ['Contact_Postal_Address_1stPart...,"{'practices': ['Contact_ZIP_1stParty', 'Contac...","{'practices': ['Demographic_1stParty', 'Demogr...","{'practices': ['Demographic_Age_1stParty', 'De...",...,"{'practices': ['Identifier_MAC_1stParty', 'Ide...",{'practices': ['Identifier_Mobile_Carrier_1stP...,{'practices': ['Identifier_SIM_Serial_1stParty...,{'practices': ['Identifier_SSID_BSSID_1stParty...,"{'practices': ['Location_1stParty', 'Location_...","{'practices': ['Location_Bluetooth_1stParty', ...","{'practices': ['Location_Cell_Tower_1stParty',...","{'practices': ['Location_GPS_1stParty', 'Locat...","{'practices': ['Location_IP_Address_1stParty',...","{'practices': ['Location_WiFi_1stParty', 'Loca..."


In [22]:
first_feature = json_normalize(data_types[0])
first_feature

,practices,features
0,"[Contact_1stParty, Contact_3rdParty]","[contact info, contact details, contact data, ..."


## Full extraction route

In [44]:
def get_list_of_practice_groups():
    """
    Requires the folder "APP_350_v1_1" to be in the same directory as this file.
    
    Returns a list where each element is a list of practices, representing a category of practices
    e.g. ['Contact_E_Mail_Address_1stParty', 'Contact_E_Mail_Address_3rdParty']
    There are 29 categories.    
    
    A full list of individual practices can then be optained with a list comprehension: 
    
    list_of_practices = [practice for practice_group in list_of_practice_groups for practice in practice_group]
    There are 58 individual practices.
   
    """
    with open("APP_350_v1_1/features.yml", "r") as stream:
        try:
            features_yml = (json_normalize(yaml.safe_load(stream)))
        except yaml.YAMLError as exc:
            print(exc)
    
    data_types = json_normalize(features_yml['data_types'])
    
    list_of_practice_groups = []
    practice_groups = range(len(data_types.columns))
    
    for i in practice_groups:
        practices_and_features = json_normalize(data_types[i])

        list_of_practice_groups.extend(practices_and_features['practices'])
    
    print(f"{len(list_of_practice_groups)} different groups of practices returned, containing {len([practice for practice_group in list_of_practice_groups for practice in practice_group])} individual practices.")
    
    return list_of_practice_groups

In [45]:
list_of_practice_groups = get_list_of_practice_groups()

29 different groups of practices returned, containing 58 individual practices.


In [46]:
list_of_practices = [practice for practice_group in list_of_practice_groups for practice in practice_group]

58

In [2]:
list_of_practice_groups = priv_pol_funcs.get_list_of_practice_groups()

29 different groups of practices returned, containing 58 individual practices.


In [3]:
list_of_practices = [practice for practice_group in list_of_practice_groups for practice in practice_group]

In [5]:
list_of_practices[:5]

['Contact_1stParty',
 'Contact_3rdParty',
 'Contact_Address_Book_1stParty',
 'Contact_Address_Book_3rdParty',
 'Contact_City_1stParty']

In [ ]:
with open("APP_350_v1_1/features.yml", "r") as stream:
    try:
        features_yml = (json_normalize(yaml.safe_load(stream)))
    except yaml.YAMLError as exc:
        print(exc)
data_types = json_normalize(features_yml['data_types'])

In [23]:
first_feature = json_normalize(data_types[0])
first_list_of_practices = first_feature['practices']
first_list_of_practices

0    [Contact_1stParty, Contact_3rdParty]
Name: practices, dtype: object

In [27]:
list_of_practices = []
m_list = [1, 2]
list_of_practices.extend(m_list)
list_of_practices.extend([1,2,3])
list_of_practices

[1, 2, 1, 2, 3]

In [35]:
list_of_practice_groups = []
practice_groups = range(len(data_types.columns))
for i in practice_groups:
    practices_and_features = json_normalize(data_types[i])
    
    list_of_practice_groups.extend(practices_and_features['practices'])

In [36]:
list_of_practice_groups

[['Contact_1stParty', 'Contact_3rdParty'],
 ['Contact_Address_Book_1stParty', 'Contact_Address_Book_3rdParty'],
 ['Contact_City_1stParty', 'Contact_City_3rdParty'],
 ['Contact_E_Mail_Address_1stParty', 'Contact_E_Mail_Address_3rdParty'],
 ['Contact_Password_1stParty', 'Contact_Password_3rdParty'],
 ['Contact_Phone_Number_1stParty', 'Contact_Phone_Number_3rdParty'],
 ['Contact_Postal_Address_1stParty', 'Contact_Postal_Address_3rdParty'],
 ['Contact_ZIP_1stParty', 'Contact_ZIP_3rdParty'],
 ['Demographic_1stParty', 'Demographic_3rdParty'],
 ['Demographic_Age_1stParty', 'Demographic_Age_3rdParty'],
 ['Demographic_Gender_1stParty', 'Demographic_Gender_3rdParty'],
 ['SSO', 'Facebook_SSO'],
 ['Identifier_1stParty', 'Identifier_3rdParty'],
 ['Identifier_Ad_ID_1stParty', 'Identifier_Ad_ID_3rdParty'],
 ['Identifier_Cookie_or_similar_Tech_1stParty',
  'Identifier_Cookie_or_similar_Tech_3rdParty'],
 ['Identifier_Device_ID_1stParty', 'Identifier_Device_ID_3rdParty'],
 ['Identifier_IMEI_1stParty', '

In [42]:
len(list_of_practice_groups)

29

In [38]:
list_of_practices = [practice for practice_group in list_of_practice_groups for practice in practice_group]

In [41]:
list_of_practices

['Contact_1stParty',
 'Contact_3rdParty',
 'Contact_Address_Book_1stParty',
 'Contact_Address_Book_3rdParty',
 'Contact_City_1stParty',
 'Contact_City_3rdParty',
 'Contact_E_Mail_Address_1stParty',
 'Contact_E_Mail_Address_3rdParty',
 'Contact_Password_1stParty',
 'Contact_Password_3rdParty',
 'Contact_Phone_Number_1stParty',
 'Contact_Phone_Number_3rdParty',
 'Contact_Postal_Address_1stParty',
 'Contact_Postal_Address_3rdParty',
 'Contact_ZIP_1stParty',
 'Contact_ZIP_3rdParty',
 'Demographic_1stParty',
 'Demographic_3rdParty',
 'Demographic_Age_1stParty',
 'Demographic_Age_3rdParty',
 'Demographic_Gender_1stParty',
 'Demographic_Gender_3rdParty',
 'SSO',
 'Facebook_SSO',
 'Identifier_1stParty',
 'Identifier_3rdParty',
 'Identifier_Ad_ID_1stParty',
 'Identifier_Ad_ID_3rdParty',
 'Identifier_Cookie_or_similar_Tech_1stParty',
 'Identifier_Cookie_or_similar_Tech_3rdParty',
 'Identifier_Device_ID_1stParty',
 'Identifier_Device_ID_3rdParty',
 'Identifier_IMEI_1stParty',
 'Identifier_IMEI_3r

## Demo EDA